## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pypsa
import pandas as pd
import sys, os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import plotting.collection as clt
from plotting.utils import load_memory, assign_carriers

In [ ]:
%matplotlib inline

## Preliminaries

In [ ]:
plt.style.use("bmh")

In [ ]:
model_names = {
    "transport": "Transport",
    "lossytransport": "Lossy\nTransport",
    "lossless-0": "Lossless",
    "lossy-3-0": "Lossy (3T)",
    "lossy-5-0": "Lossy (5T)",
    "lossless-3": "Iterative\nLossless",
    "lossy-3-3": "Iterative\nLossy (3T)",
    "lossy-5-3": "Iterative\nLossy (5T)"
}

In [ ]:
nodes = 200
opts = "2H"

In [ ]:
d = "../summary/"

if not os.path.exists(d):
    os.makedirs(d)

In [ ]:
csvargs = {"float_format": "%.3f"}

## Read Data

In [ ]:
flow_models = ["transport", "lossytransport", "lossless-0", "lossless-3", "lossy-3-0", "lossy-5-0", "lossy-3-3", "lossy-5-3"]

In [ ]:
logs = {fm: load_memory(f"../logs/elec_s_{nodes}_lcopt_{opts}_M{fm}_memory.log") for fm in flow_models}

In [ ]:
pflogs = {fm: pd.read_csv(f"../results/pf/log_elec_s_{nodes}_ec_lcopt_{opts}_M{fm}_Sdistributed.csv", index_col=0, header=[0,1]) for fm in flow_models}

In [ ]:
networks = {fm: pypsa.Network(f"../results/networks/elec_s_{nodes}_ec_lcopt_{opts}_M{fm}.nc") for fm in flow_models}

In [ ]:
networks_pf = {fm: pypsa.Network(f"../results/pf/elec_s_{nodes}_ec_lcopt_{opts}_M{fm}_Sdistributed.nc") for fm in flow_models}

In [ ]:
for n in networks.values():
    assign_carriers(n)
    if "lossy" in n.flow_model:
        n.lines_t["p0l"] = n.lines_t.p0 + n.lines_t.loss.where(n.lines_t.p0>0).fillna(0.)
        n.lines_t["p1l"] = n.lines_t.p1 + n.lines_t.loss.where(n.lines_t.p1>0).fillna(0.)
        n.lines_t["loading"] = n.lines_t.p0 + n.lines_t.loss.where(n.lines_t.p0>0).fillna(0.) - n.lines_t.loss.where(n.lines_t.p0<0).fillna(0.)
    else:
        n.lines_t["loading"] = n.lines_t.p0
        
    n.lines_t["relative_loading"] = n.lines_t.loading / n.lines.s_nom_opt

In [ ]:
for npf in networks_pf.values():
    npf.lines_t["loading"] = npf.lines_t.p0.where(npf.lines_t.p0>0).fillna(0.) - npf.lines_t.p1.where(npf.lines_t.p1>0).fillna(0.)
    npf.lines_t["relative_loading"] = npf.lines_t.loading / n.lines.s_nom_opt

## Make Tables

In [ ]:
pd.DataFrame(
    {k: clt.check_flow_errors(networks[k], networks_pf[k]) for k in networks.keys()},
    index=["mse", "rmse", "mae", "mape", "corr", "r2"],
).to_csv(f"{d}flow_errors.csv", **csvargs)

In [ ]:
pd.DataFrame({k: clt.check_capacities(v) for k, v in networks.items()}).to_csv(
    f"{d}capacity.csv", **csvargs
)

In [ ]:
pd.DataFrame({k: clt.check_energy_generated(v) for k, v in networks.items()}).to_csv(
    f"{d}energy.csv", **csvargs
)

In [ ]:
pd.DataFrame({k: clt.check_slack(v, pflogs[k]) for k, v in networks_pf.items()}).to_csv(
    f"{d}slack.csv", **csvargs
)

In [ ]:
pd.DataFrame(
    {
        k: [
            clt.check_curtailment(v),
            clt.check_energy_transmitted(v),
            *clt.check_costs(v),
            *clt.check_energy_balance(v),
        ]
        for k, v in networks.items()
    },
    index=[
        "curtailment",
        "energy-transmitted",
        "absolute-tsc",
        "relative-tsc",
        "energy-balance",
        "loss-share",
        "link-losses",
        "line-losses",
    ],
).to_csv(f"{d}misc.csv", **csvargs)

## Make Plots (Multiple Models)

In [ ]:
clt.plot_performance(logs, 'peak_mem', model_names, colors="forestgreen", fn=f"{d}memory.pdf")
clt.plot_performance(logs, 'time', model_names, colors="midnightblue", fn=f"{d}time.pdf")

In [ ]:
c = ['firebrick', 'indianred', 'midnightblue', 'darkmagenta', "darkorange", "orange", "forestgreen", "limegreen"]
clt.plot_performance(logs, 'peak_mem', model_names, colors=c, fn=f"{d}memory-c.pdf")
clt.plot_performance(logs, 'time', model_names, colors=c, fn=f"{d}time-c.pdf")

In [ ]:
groups = {
    "Generator": ["onwind", "offwind-ac", "offwind-dc", "solar"],
    "StorageUnit": ["H2", "battery"],
    "Link": [""],
    "Line": [""]
}

for c, carriers in groups.items():
    for carrier in carriers:
        clt.plot_capacity_correlation(networks, c, model_names, regex=carrier, fn=f"{d}capacity-correlation-{c}-{carrier}.pdf")

In [ ]:
conv_share = pd.concat([clt.convergence_share(pflogs[fm], fm) for fm in flow_models], axis=1)
conv_share.to_csv(f"{d}power-flow-convergence.csv")
clt.plot_nonconverged(conv_share, model_names, fn=f"{d}power-flow-convergence.pdf")

In [ ]:
clt.plot_cost_bar(networks, model_names, fn=f"{d}cost-bar.pdf")

## Make Plots (Individual Models)

In [ ]:
clt.plot_feasible_space(networks["transport"].lines.iloc[2], fn=f"{d}feasiblespace.pdf")

In [ ]:
for name in networks.keys():
    
    print(name)
    
    n = networks[name]
    n_pf = networks_pf[name]
    
    clt.plot_network(n, fn=f"{d}network-{name}.pdf")
    clt.plot_negative_marginal_prices(n, fn=f"{d}negative-lmp-{name}.pdf")
    
    clt.plot_flow_comparison(n, n_pf, fn=f"{d}flow-comparison-{name}.pdf")
    
    clt.plot_duration_curve(n, n_pf, apparent=False, fn=f"{d}flow-duration-curve-p-{name}.pdf")
    clt.plot_duration_curve(n, n_pf, apparent=True, fn=f"{d}flow-duration-curve-s-{name}.pdf")
    
    clt.plot_v_ang_diff(n_pf, fn=f"{d}v-ang-diff-{name}.pdf")
    
    if "lossy" in name:
        clt.plot_network_losses(n, fn = f"{d}network-losses-{name}.pdf")
        clt.plot_flow_vs_loss(n, fn=f"{d}flow-vs-loss-{name}.pdf")
        clt.plot_loss_comparison(n, n_pf, fn=f"{d}loss-comparison-{name}.pdf")